# Using `cclib` to parse quantum chemical outputs

Running a calculation is pointless unless we can access the data. It is not only incredibly boring, but error-prone and an innefficient use of time to manually extract data from output files.

The process of reading an output file and transforming the raw data contained within into information is called parsing. The prospect of writing a program to parse output files from scratch, whilst exhilarating, is daunting.
Luckily, a lot of people have already done a lot of work for us.

The python module [`cclib`](https://www.github.com/cclib/cclib "cclib on github") is able to parse a vast number of properties from a wide array of commonly used quantum chemical software, including `Gaussian`, `ORCA`, `Molpro`, `Q-Chem`, and others. A complete list of available properties and compatible software can be found [here](https://cclib.github.io/data.html "Data & software parsed by cclib").

## Obtaining the package
`cclib` is not provided in the Anaconda default installation. We must install it ourselves manually.

Unfortunately, there is a bug in the current release of `cclib` that means we need to install a slightly older version of it, version `1.5.3`

##### Installing with Anaconda Navigator
Follow the instructions on [this page](https://docs.anaconda.com/anaconda/navigator/tutorials/manage-packages/ "Anaconda Navigator tutorial"), making sure to install `cclib 1.5.3` specifically.

##### Installing with pip
We can use the command
`pip install cclib==1.5.3`
alternatively, to obtain the current development version, in which the issue has also been solved
`pip install git+https://github.com/cclib/cclib`

## The problem at hand: H atom dissociation in phenol

In this notebook, we will use `cclib` to plot the dissociation of the OH bond in phenol. I have provided output files from calculations performed using [`ORCA 4.0.1.2`](https://orcaforum.cec.mpg.de/ "ORCA quantum chemistry program homepage") in the directory `phenol_dissoc_outputs`, which you can read for yourself if you are interested.

**The aim of this workshop is to plot a potential energy curve for H atom dissociation in phenol.**

As we will see, `cclib` will make the analysis of our calculations painless. First, however, we need to get a list of the files we want to process. We will use the in built module `os` to do this.

Follow the instructions below (in **bold**). Please try to retain the variable names I have defined, since this will make debugging easier for us all.

In [ ]:
import os

###  Finding the text files we want to parse

Our first task is to make a list of output files. We will do this with a `for` loop.

The directory containing the files is called `phenol_dissoc_outputs` and all of the files end with `.out`

**Write a loop which puts all of the filenames in a list**

In [ ]:
file_list = []

orca_file_dir = 'phenol_dissoc_outputs'
for file in os.listdir(orca_file_dir):
    if file.endswith(".out"):
        file_list.append(file)


### Parsing the files using `cclib`
Now that we have a list of files, we need to actually parse them.

The recommended way to do this is using the `cclib.io.ccread()` function

**Import cclib and write a loop that takes the filename list and parses the files**

In [ ]:
parsed_files = []

import cclib

for file in file_list:
    parsed_files.append(cclib.io.ccread('{}/{}'.format(orca_file_dir, file)))

# keep this variable to make future examples easier
example_file = parsed_files[0]

### Getting data from a parsed file
Parsing a text file with `cclib` creates a new object. The methods of this object return data which has been extracted from the file.

We can find a list of the data which can be extracted by `cclib` [here](https://cclib.github.io/data.html "data extracted with cclib"). This tells us the name of the method which returns given data. For example, an array of the solved SCF energies from calculations can be obtained by using the `scfenergies` method.

In [ ]:
example_energy = example_file.scfenergies[0]
print('{}'.format(example_energy))

We can find the available methods of a specific object with the following trick

In [ ]:
available_methods = example_file.__dict__.keys()
print('{}'.format(available_methods))

### Collecting the energies
Now that we have a list of parsed files, we can now use the `scfenergies` method to obtain a list containing all of the ground state energies.

**Write a loop using the `scfenergies` method to extract the ground state energies**

In [ ]:
gs_energies = []
for file in parsed_files:
    gs_energies.append(file.scfenergies)


### Obtaining the bond lengths
We now have an array containing the energies. This will form our y-axis data. We now need to obtain the x-axis data, _i.e._ the bond lengths at each geometry.

We can also obtain the molecular geometries extracted from an output file using `cclib`. In this case, we will use the `atomcoords` method. 

Note that the dimensionality of the array we obtain from `atomcoords` is not the same as with `scfenergies`.

In [ ]:
example_geometry = example_file.atomcoords[0]
print('{}'.format(example_geometry))

Unfortunately, there is no easy way to visualise the output file directly in this notebook. For now, just note that in all of the calculations, atoms 12 and 13 are the OH pair. 

In [ ]:
example_O_coord = example_geometry[11]
example_H_coord = example_geometry[12]
print('O xyz:')
print('{}'.format(example_O_coord))
print('H xyz:')
print('{}'.format(example_H_coord))

### Using `numpy` to do mathematics

The `numpy` module provides convenient implementations of many mathematical operations. We will use it to get the square root of our number.

**Use the pythagorean theorem to calculate the distance between O and H in this example**

In [ ]:
import numpy as np

example_distance = 0
for i in range(len(example_O_coord)):
    example_distance += example_H_coord[i]**2 - example_O_coord[i]**2
example_distance = np.sqrt(example_distance)

print('O-H bond length: {:.3f} Å'.format(example_distance))

**Write a loop to obtain the OH distances for all of the files**

In [ ]:
bond_lengths = []
for file in parsed_files:
    O_xyz = file.atomcoords[0][11]
    H_xyz = file.atomcoords[0][12]
    bond_lengths.append(np.linalg.norm(H_xyz - O_xyz))

### Plotting our data using `matplotlib`

We now have everything we need to plot the results. We will be using `matplotlib.pyplot` to do this.

We will import `matplotlib.pyplot` and change some settings to make it look better in this notebook

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

**Prepare a plotting environment, create a scatter plot object, then display the plot**

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('O-H bond length / Å')
ax.set_ylabel('Absolute energy / eV')
ax.scatter(bond_lengths, gs_energies)
plt.show()  # this should be your final command 

## Additional work

I will list some further challenges below:

**Use `cclib.parser.utils.convertor` to alter the units:**

In [ ]:
import cclib.parser.utils as utils

gs_energies_au = [utils.convertor(energy, "eV", "hartree") for energy in gs_energies]

fig, ax = plt.subplots()
ax.set_xlabel('O-H bond length / Å')
ax.set_ylabel('Absolute energy / a.u.')
ax.scatter(bond_lengths, gs_energies_au)
plt.show()

**Rather than the absolute energies, plot the relative energies:**

In [ ]:
relative_gs_energies_au = gs_energies_au - np.min(gs_energies_au)

fig, ax = plt.subplots()
ax.set_xlabel('O-H bond length / Å')
ax.set_ylabel('Relative energy / a.u.')
ax.scatter(bond_lengths, relative_gs_energies_au)
plt.show()

**The calculations also include excited states. Include these in the plot.**

In [ ]:
relative_es_energies_au = []
for file in parsed_files:
    scf = utils.convertor(file.scfenergies[0], "eV", "hartree")
    exe = utils.convertor(file.etenergies, "wavenumber", "hartree")
    relative_es_energies_au.append(scf + exe - np.min(gs_energies_au))
relative_es_energies_au = np.array(relative_es_energies_au)

fig, ax = plt.subplots()
ax.set_xlabel('O-H bond length / Å')
ax.set_ylabel('Relative energy / a.u.')
ax.scatter(bond_lengths, relative_gs_energies_au)
for i in range(len(relative_es_energies_au[0])):
    ax.scatter(bond_lengths, [relative_es_energies_au[:, i]])
plt.show()